In [ ]:
import requests
import pandas as pd
import time

# --- НАСТРОЙКИ ---
# Ваш персональный токен (уже вставлен)
TOKEN = 'input your token here'
HEADERS = {"Authorization": f"token {TOKEN}"}
MAX_REPOS = 1000
REPOS_PER_PAGE = 100

def get_top_repositories():
    all_repos = []
    print("--- Этап 1: Сбор метаданных ТОП-1000 репозиториев ---")

    # Идем по страницам (GitHub Search API отдает максимум 1000 результатов)
    for page in range(1, 11):
        # Запрос: репозитории со звездами > 10000, сортировка по звездам
        url = f"https://api.github.com/search/repositories?q=stars:>10000&sort=stars&order=desc&per_page={REPOS_PER_PAGE}&page={page}"
        response = requests.get(url, headers=HEADERS)

        if response.status_code == 200:
            items = response.json().get('items', [])
            all_repos.extend(items)
            print(f"Загружена страница {page}/10. Всего собрано: {len(all_repos)}")
        else:
            print(f"Ошибка на странице {page}: {response.status_code}")
            print(response.json())
            break

        # Search API очень чувствителен к лимитам, ждем между страницами
        time.sleep(5)

    return all_repos

def analyze_repo_structure(repo):
    full_name = repo['full_name']
    url_contents = f"https://api.github.com/repos/{full_name}/contents/"

    res = requests.get(url_contents, headers=HEADERS)

    # Инициализируем теги (правила)
    has_coc = False
    has_contributing = False
    has_workflows = False
    has_readme = False

    if res.status_code == 200:
        content_list = res.json()
        filenames = [f['name'].upper() for f in content_list]

        # Проверяем наличие ключевых файлов
        has_coc = any("CODE_OF_CONDUCT" in name for name in filenames)
        has_contributing = any("CONTRIBUTING" in name for name in filenames)
        has_readme = any("README" in name for name in filenames)
        has_workflows = ".GITHUB" in filenames # Обычно workflows внутри этой папки

    return {
        'repo_name': repo['name'],
        'organization': full_name.split('/')[0],
        'language': repo['language'],
        'stars': repo['stargazers_count'],
        'forks': repo['forks_count'],
        'open_issues': repo['open_issues_count'],
        'age_days': (pd.Timestamp.now(tz='UTC') - pd.to_datetime(repo['created_at'])).days,
        'has_coc': has_coc,
        'has_contributing': has_contributing,
        'has_workflows': has_workflows,
        'has_readme': has_readme,
        'description': repo['description']
    }

# --- ОСНОВНОЙ ЦИКЛ ---
top_repos = get_top_repositories()
final_results = []

print("\n--- Этап 2: Глубокий анализ структуры каждого репозитория ---")
print("Это займет время (около 30-40 минут) из-за лимитов GitHub API.")

for i, repo in enumerate(top_repos):
    try:
        data = analyze_repo_structure(repo)
        final_results.append(data)

        if (i + 1) % 50 == 0:
            print(f"Обработано {i+1} из {len(top_repos)}...")

        # Пауза, чтобы не превысить лимит 5000 запросов в час для токена
        # и лимиты на просмотр содержимого
        time.sleep(1.5)

    except Exception as e:
        print(f"Ошибка при обработке {repo['full_name']}: {e}")

# Сохранение в CSV
df = pd.DataFrame(final_results)
df.to_csv('top_1000_os_rules.csv', index=False, encoding='utf-8')

print("\n--- АНАЛИЗ ЗАВЕРШЕН ---")
print(f"Файл 'top_1000_os_rules.csv' готов. Всего записей: {len(df)}")